### Importing dataset and putting into a dataframe

In [102]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
my_iris = load_iris()

In [51]:
dir(my_iris)

['DESCR', 'data', 'feature_names', 'filename', 'target', 'target_names']

In [52]:
df_iris = pd.DataFrame(my_iris.data, columns = my_iris.feature_names)
df_iris.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [53]:
df_iris['flower'] = my_iris.target
df_iris.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),flower
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [54]:
X = df_iris.drop(['flower'],axis='columns')
y = df_iris.flower

In [55]:
print(X)
print(y)

     sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
0                  5.1               3.5                1.4               0.2
1                  4.9               3.0                1.4               0.2
2                  4.7               3.2                1.3               0.2
3                  4.6               3.1                1.5               0.2
4                  5.0               3.6                1.4               0.2
..                 ...               ...                ...               ...
145                6.7               3.0                5.2               2.3
146                6.3               2.5                5.0               1.9
147                6.5               3.0                5.2               2.0
148                6.2               3.4                5.4               2.3
149                5.9               3.0                5.1               1.8

[150 rows x 4 columns]
0      0
1      0
2      0
3      0
4   

In [56]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3)

In [57]:
print(len(X_train), len(X_test))

105 45


### Randomly varying parameters

In [58]:
from sklearn.svm import SVC
model_iris_svm = SVC(kernel = 'rbf', C=30,gamma='auto')
model_iris_svm.fit(X_train,y_train)

SVC(C=30, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [59]:
model_iris_svm.score(X_test,y_test)

0.9555555555555556

In [60]:
from sklearn.model_selection import cross_val_score

In [61]:
cross_val_score(SVC(kernel='linear',C=10,gamma='auto'),X,y,cv=5)

array([1.        , 1.        , 0.9       , 0.96666667, 1.        ])

In [62]:
cross_val_score(SVC(kernel='rbf',C=10,gamma='auto'),X,y,cv=5)

array([0.96666667, 1.        , 0.96666667, 0.96666667, 1.        ])

In [63]:
cross_val_score(SVC(kernel='rbf',C=20,gamma='auto'),X,y,cv=5)

array([0.96666667, 1.        , 0.9       , 0.96666667, 1.        ])

### Using a for loop

In [64]:
kernels = ['rbf','linear']
C = [1,10,20]
avg_scores = {}
for my_kernel in kernels:
    for my_C in C:
        cv_scores = cross_val_score(SVC(kernel=my_kernel, C=my_C,gamma='auto'),X,y,cv=5)
        avg_scores[my_kernel+"_"+str(my_C)] = np.average(cv_scores)
avg_scores

{'rbf_1': 0.9800000000000001,
 'rbf_10': 0.9800000000000001,
 'rbf_20': 0.9666666666666668,
 'linear_1': 0.9800000000000001,
 'linear_10': 0.9733333333333334,
 'linear_20': 0.9666666666666666}

### Using GridSearchCV

In [65]:
from sklearn.model_selection import GridSearchCV

clf = GridSearchCV(SVC(),{
    'C':[1,10,20,50],
    'kernel':['rbf','linear']
},cv=5,return_train_score=False)

In [66]:
clf.fit(X,y)
clf.cv_results_

{'mean_fit_time': array([0.00306425, 0.00293131, 0.002074  , 0.00235271, 0.00189419,
        0.00185242, 0.0018877 , 0.00283318]),
 'std_fit_time': array([6.06715214e-04, 8.28722615e-04, 1.25577490e-04, 9.07700143e-04,
        4.23562936e-05, 3.71861451e-05, 2.04421374e-05, 8.07226067e-04]),
 'mean_score_time': array([0.00123653, 0.00164938, 0.00092444, 0.00087028, 0.00084968,
        0.00088067, 0.00084481, 0.0014421 ]),
 'std_score_time': array([9.68563674e-05, 3.60471735e-04, 6.75017863e-05, 1.49732942e-05,
        2.20050920e-05, 8.72254168e-05, 1.18651037e-05, 3.28893244e-04]),
 'param_C': masked_array(data=[1, 1, 10, 10, 20, 20, 50, 50],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear',
                    'rbf', 'linear'],
              mask=[False, False, False, False, False, False, False, False],
        fill_valu

In [67]:
df_results = pd.DataFrame(clf.cv_results_)
df_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.003064,0.000607,0.001237,0.000097,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.966667,0.966667,0.966667,0.933333,1.0,0.966667,0.021082,6
1,0.002931,0.000829,0.001649,0.000360,1,linear,"{'C': 1, 'kernel': 'linear'}",0.966667,1.000000,0.966667,0.966667,1.0,0.980000,0.016330,1
2,0.002074,0.000126,0.000924,0.000068,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.966667,1.000000,0.966667,0.966667,1.0,0.980000,0.016330,1
3,0.002353,0.000908,0.000870,0.000015,10,linear,"{'C': 10, 'kernel': 'linear'}",1.000000,1.000000,0.900000,0.966667,1.0,0.973333,0.038873,4
4,0.001894,0.000042,0.000850,0.000022,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.966667,1.000000,0.966667,0.966667,1.0,0.980000,0.016330,1
5,0.001852,0.000037,0.000881,0.000087,20,linear,"{'C': 20, 'kernel': 'linear'}",1.000000,1.000000,0.900000,0.933333,1.0,0.966667,0.042164,6
6,0.001888,0.000020,0.000845,0.000012,50,rbf,"{'C': 50, 'kernel': 'rbf'}",0.966667,1.000000,0.933333,0.966667,1.0,0.973333,0.024944,4
7,0.002833,0.000807,0.001442,0.000329,50,linear,"{'C': 50, 'kernel': 'linear'}",1.000000,1.000000,0.900000,0.933333,1.0,0.966667,0.042164,6


In [68]:
df_my_results = df_results[['param_C','param_kernel','mean_test_score']]
df_my_results

,param_C,param_kernel,mean_test_score
0,1,rbf,0.966667
1,1,linear,0.980000
2,10,rbf,0.980000
3,10,linear,0.973333
4,20,rbf,0.980000
5,20,linear,0.966667
6,50,rbf,0.973333
7,50,linear,0.966667


In [69]:
dir(clf)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_check_is_fitted',
 '_estimator_type',
 '_format_results',
 '_get_param_names',
 '_get_tags',
 '_more_tags',
 '_pairwise',
 '_required_parameters',
 '_run_search',
 'best_estimator_',
 'best_index_',
 'best_params_',
 'best_score_',
 'classes_',
 'cv',
 'cv_results_',
 'decision_function',
 'error_score',
 'estimator',
 'fit',
 'get_params',
 'iid',
 'inverse_transform',
 'multimetric_',
 'n_jobs',
 'n_splits_',
 'param_grid',
 'pre_dispatch',
 'predict',
 'predict_log_proba',
 'predict_proba',
 'refit',
 'refit_time_',
 'return_train_score',
 'score

In [70]:
clf.best_score_

0.9800000000000001

In [71]:
clf.best_params_

{'C': 1, 'kernel': 'linear'}

### Using RandomizedSearchCV 

In [93]:
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(SVC(gamma='auto'),
                       {'C':[1,10,20] , 'kernel':['rbf','linear']},
                        cv=5,
                        return_train_score=False,
                        n_iter=2)

In [94]:
rs.fit(X,y)

RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                                 class_weight=None, coef0=0.0,
                                 decision_function_shape='ovr', degree=3,
                                 gamma='auto', kernel='rbf', max_iter=-1,
                                 probability=False, random_state=None,
                                 shrinking=True, tol=0.001, verbose=False),
                   iid='deprecated', n_iter=2, n_jobs=None,
                   param_distributions={'C': [1, 10, 20],
                                        'kernel': ['rbf', 'linear']},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False, scoring=None, verbose=0)

In [95]:
my_results = pd.DataFrame(rs.cv_results_)[['param_C','param_kernel','mean_test_score']]
my_results

,param_C,param_kernel,mean_test_score
0,1,linear,0.980000
1,20,rbf,0.966667


### Choosing Best ML Model

In [104]:
my_models = {
    'svm':{
            'model':SVC(gamma='auto'),
            'params':{'C':[1,10,20],'kernel':['rbf','linear']}
            },
    'random_forest':{
            'model':RandomForestClassifier(),
            'params': {'n_estimators':[1,5,10]}
            },
    'logistic_regression':{
            'model':LogisticRegression(solver='liblinear',multi_class='auto'),
            'params': {'C':[1,5,10]}
            }
}

In [109]:
scores = []

for model_name, model_params in my_models.items():
    clf = GridSearchCV(model_params['model'], 
                       model_params['params'],
                       cv=5,
                       return_train_score=False)
    clf.fit(X,y)
    scores.append({
        'model': model_name,
        'best_score':clf.best_score_,
        'best_params':clf.best_params_,
    })
    
pd.DataFrame(scores)

,model,best_score,best_params
0,svm,0.980000,"{'C': 1, 'kernel': 'rbf'}"
1,random_forest,0.960000,{'n_estimators': 5}
2,logistic_regression,0.966667,{'C': 5}
